In [1]:
import finetune_resnet
import h5py
from keras.metrics import sparse_categorical_accuracy
from keras.optimizers import SGD
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt

C:\tools\miniconda3\lib\site-packages\h5py\__init__.py:75: UserWarning: h5py is running against HDF5 1.10.5 when it was built against 1.10.4, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [2]:
dataset = h5py.File("../Market-1501/market.h5")
x = dataset["train_images"]
attributes_target = dataset["train_labels"]
file_names = dataset["train_images_files"]
ids_encoder = dict()
train_ids = np.zeros(len(file_names), dtype=int)
i = 0
for j, file in enumerate(file_names):
    if file[:4] not in ids_encoder:
        ids_encoder[file[:4]] = i
        i += 1
    train_ids[j] = ids_encoder[file[:4]]
n_id = len(ids_encoder)

In [3]:
train_ids.shape, n_id

((12567,), 751)

In [4]:
_, training_model, inference_model = finetune_resnet.get_models(n_person=n_id)














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
training_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 64, 3)   0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 2048)         23587712    input_1[0][0]                    
__________________________________________________________________________________________________
attributes (Dense)              (None, 27)           55323       resnet50[1][0]                   
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 512)          1049088     resnet50[1][0]                   
__________________________________________________________________________________________________
dense_1 (D

In [6]:
# sparse_top_k_categorical_accuracy

lam = 0.9
def attr_loss(y_true, y_pred):
    return lam * finetune_resnet.market_attribute_accuracy(y_true, y_pred)
def id_loss(y_true, y_pred):
    return (1 - lam) * sparse_categorical_accuracy(y_true, y_pred)
# optim -> SGD
optim = SGD()
training_model.compile("adam", loss=["binary_crossentropy", "sparse_categorical_crossentropy"], metrics={"attributes":[finetune_resnet.market_attribute_accuracy], "ids":[sparse_categorical_accuracy]})



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
attributes, categorical_id = training_model.predict(finetune_resnet.preprocess_images(x[:1]))
attributes.shape, categorical_id.shape

((1, 27), (1, 751))

In [8]:
id_embedding = inference_model.predict(finetune_resnet.preprocess_images(x[:1]))
id_embedding.shape

(1, 539)

In [9]:
training_model.load_weights("../models/full-model-weights-14.37.h5")
cb = [
    ModelCheckpoint("../models/full-model-weights-{epoch:02d}.h5", save_weights_only=True),
    EarlyStopping(monitor="loss")
]

# n_samples = 400
epochs = 20
batch_size = 32

history = training_model.fit(
    x=finetune_resnet.preprocess_images(np.array(x)),
    y=[finetune_resnet.preprocess_labels(np.array(attributes_target)), train_ids],
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
    callbacks=cb,
    initial_epoch=7
)



Train on 11310 samples, validate on 1257 samples
Epoch 8/20
11310/11310 [==============================] - 2002s 177ms/step - loss: 0.4467 - attributes_loss: 0.1129 - ids_loss: 0.3338 - attributes_market_attribute_accuracy: 0.9154 - ids_sparse_categorical_accuracy: 0.9119 - val_loss: 13.6304 - val_attributes_loss: 0.2702 - val_ids_loss: 13.3602 - val_attributes_market_attribute_accuracy: 0.8211 - val_ids_sparse_categorical_accuracy: 7.9554e-04
Epoch 9/20
11310/11310 [==============================] - 1935s 171ms/step - loss: 0.3395 - attributes_loss: 0.1015 - ids_loss: 0.2380 - attributes_market_attribute_accuracy: 0.9237 - ids_sparse_categorical_accuracy: 0.9348 - val_loss: 14.7314 - val_attributes_loss: 0.2674 - val_ids_loss: 14.4639 - val_attributes_market_attribute_accuracy: 0.8277 - val_ids_sparse_categorical_accuracy: 0.0032
Epoch 10/20
11310/11310 [==============================] - 1947s 172ms/step - loss: 0.2688 - attributes_loss: 0.0905 - ids_loss: 0.1783 - attributes_market

KeyboardInterrupt: 

In [10]:
history.history

NameError: name 'history' is not defined

In [11]:
# test function with query - gallery
test_data = np.array([
    x[0], x[1], x[2], x[100], x[200]
])
res = inference_model.predict(finetune_resnet.preprocess_images(test_data))

In [12]:
# 0 1 2 are the same person, 3, 4 are different persons
dist = [
    np.linalg.norm(res[0] - res[1]),
    np.linalg.norm(res[0] - res[2]),
    np.linalg.norm(res[0] - res[3]),
    np.linalg.norm(res[0] - res[4]),
]
dist

[20.43938, 20.798864, 41.726013, 39.351517]

In [13]:
# test on unseen data
test_data = np.array([
    x[400], x[401], x[402], x[500], x[600]
])
res = inference_model.predict(finetune_resnet.preprocess_images(test_data))
# 0 1 2 are the same person, 3, 4 are different persons
dist = [
    np.linalg.norm(res[0] - res[1]),
    np.linalg.norm(res[0] - res[2]),
    np.linalg.norm(res[0] - res[3]),
    np.linalg.norm(res[0] - res[4]),
]
dist, (file_names[400], file_names[401], file_names[402], file_names[500], file_names[600])

([23.436071, 21.682852, 33.515865, 40.13437],
 ('0067_c5s1_009476_00.jpg',
  '0067_c5s1_019276_00.jpg',
  '0067_c5s1_019301_00.jpg',
  '0077_c4s1_018376_00.jpg',
  '0095_c4s1_015376_00.jpg'))

In [17]:
training_model.save_weights("../models/full-model-weights.h5")